### This is where I scrape local leaders and get their
#### 1. Name
#### 2. Link to resume
#### 3. Province
#### 4. Title

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', 500)

import re

In [4]:
driver = webdriver.Chrome()

In [5]:
#first, get url of each province, save as csv
driver.get("http://ldzl.people.com.cn/dfzlk/front/personProvince1.htm")

In [6]:
provinces = driver.find_elements_by_class_name("fl")[1].find_elements_by_tag_name("li")

province_list = []
for province in provinces[:-1]:
    province_dict = {}
    province_dict["link"] = province.find_element_by_tag_name("a").get_attribute("href")
    province_dict["province"] = province.text
    province_list.append(province_dict)
    
pd.DataFrame(province_list).to_csv("province.csv", index = False)

In [7]:
## I wrote some code to selenium to each leader's info
## but bs will prob do a better job here
## so, switching gears

In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
raw_html = requests.get("http://ldzl.people.com.cn/dfzlk/front/personProvince1.htm").content
info = BeautifulSoup(raw_html, "html.parser")

In [10]:
top_four = info.find("div", attrs={"class":"box01"}).find_all("dd")

for pp in top_four:
    title = pp.find("i").text
    name = pp.find("a").text
    link = "http://ldzl.people.com.cn/dfzlk/front/" + pp.find("a")["href"]

In [11]:
party_leader = info.find_all("div", attrs={"class":"box02"})[0].find_all("a")

for pp in party_leader:
    name = pp.text
    link = "http://ldzl.people.com.cn/dfzlk/front/" + pp["href"]
    title = pp.find_previous("i", attrs={"class":"red"}).text

In [12]:
admin_leader = info.find_all("div", attrs={"class":"box02"})[1].find_all("a")

for pp in admin_leader:
    name = pp.text
    link = "http://ldzl.people.com.cn/dfzlk/front/" + pp["href"]
    title = pp.find_previous("i", attrs={"class":"red"}).text

In [13]:
district_leader = info.find("div", attrs={"class":"box03"}).find_all("li")

for pp in district_leader:
    district = pp.find_all("i")[0].text
    party_secretary = pp.find_all("a")[0].text
    party_secretary_link = "http://ldzl.people.com.cn/dfzlk/front/" + pp.find_all("a")[0]["href"]
    mayor = pp.find_all("a")[1].text
    mayor_link = "http://ldzl.people.com.cn/dfzlk/front/" + pp.find_all("a")[1]["href"]

In [4]:
#combine everything together
def get_leader_info(row):
    url = row["link"]
    raw_html = requests.get(url).content
    info = BeautifulSoup(raw_html, "html.parser")
    

    try:
        top_four = info.find("div", attrs={"class":"box01"}).find_all("dd")

        for pp in top_four:
            leader_dict = {}

            title = pp.find("i").text
            name = pp.find("a").text
            link = "http://ldzl.people.com.cn/dfzlk/front/" + pp.find("a")["href"]
            leader_dict["group"] = "top four"
            leader_dict["name"] = name
            leader_dict["title"] = title
            leader_dict["link"] = link
            leader_dict["province"] = row["province"]
            leader_list.append(leader_dict)
    except:
        print(row["province"], "has no top four")

    try:
        party_leader = info.find_all("div", attrs={"class":"box02"})[0].find_all("a")

        for pp in party_leader:
            leader_dict = {}
            name = pp.text
            link = "http://ldzl.people.com.cn/dfzlk/front/" + pp["href"]
            title = pp.find_previous("i", attrs={"class":"red"}).text
            leader_dict["group"] = "party leader"
            leader_dict["name"] = name
            leader_dict["title"] = title
            leader_dict["link"] = link
            leader_dict["province"] = row["province"]
            leader_list.append(leader_dict)

    except:
        print(row["province"], "has no party leader")

    try:
        admin_leader = info.find_all("div", attrs={"class":"box02"})[1].find_all("a")

        for pp in admin_leader:
            leader_dict = {}
            name = pp.text
            link = "http://ldzl.people.com.cn/dfzlk/front/" + pp["href"]
            title = pp.find_previous("i", attrs={"class":"red"}).text
            leader_dict["group"] = "admin leader"
            leader_dict["name"] = name
            leader_dict["title"] = title
            leader_dict["link"] = link
            leader_dict["province"] = row["province"]
            leader_list.append(leader_dict)
    except:
        print(row["province"], "has no admin leader")

    try:
        district_leader = info.find("div", attrs={"class":"box03"}).find_all("li")

        for pp in district_leader:
            leader_dict = {}
            district = pp.find_all("i")[0].text
            party_secretary = pp.find_all("a")[0].text
            party_secretary_link = "http://ldzl.people.com.cn/dfzlk/front/" + pp.find_all("a")[0]["href"]
            leader_dict["group"] = "district leader"
            leader_dict["name"] = party_secretary
            leader_dict["title"] = "书记"
            leader_dict["link"] = party_secretary_link
            leader_dict["province"] = row["province"]
            leader_list.append(leader_dict)

        for pp in district_leader:
            leader_dict = {}
            district = pp.find_all("i")[0].text
            mayor = pp.find_all("a")[1].text
            mayor_link = "http://ldzl.people.com.cn/dfzlk/front/" + pp.find_all("a")[1]["href"]
            leader_dict["group"] = "district leader"
            leader_dict["name"] = mayor
            leader_dict["title"] = "市（区、州）长行署专员"
            leader_dict["link"] = mayor_link
            leader_dict["province"] = row["province"]
            leader_list.append(leader_dict)
    except:
        print(row["province"], "has no district leader")

In [5]:
leader_list = []

provinces = pd.read_csv("province.csv")

In [6]:
provinces.apply(get_leader_info, axis = 1)

香港 has no party leader
香港 has no admin leader
香港 has no district leader
澳门 has no party leader
澳门 has no admin leader
澳门 has no district leader


0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
20    None
21    None
22    None
23    None
24    None
25    None
26    None
27    None
28    None
29    None
30    None
31    None
32    None
dtype: object

In [7]:
final = pd.DataFrame(leader_list)

In [8]:
final.to_csv("local_leader_links.csv", index = False)